In [39]:
from langchain_chroma import Chroma
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
from langchain_openai import AzureOpenAIEmbeddings
import os
from langgraph.checkpoint.memory import MemorySaver

load_dotenv()

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_LLM_DEPLOYMENT"), 
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    )

embedding = AzureOpenAIEmbeddings(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT"), 
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

In [49]:
import os
if os.path.exists("/app/artifacts/embeddings/20250613_124237"):
    persist_directory = "/app/artifacts/Vector_databases/biology"
else:
    persist_directory = r"R:\MailMate\artifacts\embeddings\20250613_125644"

vectorstore = Chroma(
    collection_name="resume",
    embedding_function=embedding,
    persist_directory=persist_directory,
)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 20}
)

In [50]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, ToolMessage
from operator import add as add_messages
from langchain_core.tools import tool 

In [51]:
@tool
def retriever_tool(query: str) -> str:
    """
    This tool searches and returns the information from the document.
    """
    
    docs = retriever.invoke(query)
    
    if not docs:
        return "I found no relevant information in the document."
    
    results = []
    for i, doc in enumerate(docs):
        results.append(f"Document {i+1}:\n{doc.page_content}")
    
    return "\n\n".join(results)

In [58]:
tools = [retriever_tool]

llm = llm.bind_tools(tools)

In [59]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [60]:
def should_continue(state: AgentState):
    """Check if the last message contains tool calls."""
    result = state['messages'][-1]
    return hasattr(result, 'tool_calls') and len(result.tool_calls) > 0

In [61]:
system_prompt = """
    you are TAZMIC, a research assistant specialized in providing information from a **document**, and provide concise and accurate response in **friendly** and **formal** manner to user queries only based on the content of the document.
    
    If you do not have enough information to answer the question, you should say "I don't know" or "I found no relevant information in the document." instead of making up an answer.
"""

tools_dict = {our_tool.name: our_tool for our_tool in tools}
tools_dict

def call_llm(state: AgentState) -> AgentState:
    """Function to call the LLM with the current state."""
    messages = list(state['messages'])
    messages = [SystemMessage(content=system_prompt)] + messages
    message = llm.invoke(messages)
    return {'messages': [message]}

# Retriever Agent
def take_action(state: AgentState) -> AgentState:
    """Execute tool calls from the LLM's response."""
    
    tool_calls = state['messages'][-1].tool_calls
    results = []
    
    for t in tool_calls:
        print(f"Calling Tool: {t['name']} with query: {t['args'].get('query', 'No query provided')}")
        
        try:
            if t['name'] not in tools_dict:
                print(f"\nTool: {t['name']} does not exist.")
                result = "Incorrect Tool Name, Please Retry and Select tool from List of Available tools."
            else:
                result = tools_dict[t['name']].invoke(t['args'].get('query', ''))
                print(f"Result length: {len(str(result))}")
                
            # Always append the Tool Message with proper ID
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
            
        except Exception as e:
            print(f"Error in tool execution: {e}")
            # Even on error, we must respond to the tool call
            error_result = f"Tool execution failed: {str(e)}"
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=error_result))

    print("Tools Execution Complete. Back to the model!")
    return {'messages': results}

In [56]:
graph = StateGraph(AgentState)
graph.add_node("llm", call_llm)
graph.add_node("retriever_agent", take_action)

graph.add_conditional_edges(
    "llm",
    should_continue,
    {True: "retriever_agent", False: END}
)
graph.add_edge("retriever_agent", "llm")
graph.set_entry_point("llm")

rag_agent = graph.compile()


def query_agent(user_input: str) -> str:
    """
    Process a single user query and return the response.
    """

    messages = [HumanMessage(content=user_input)]
    
    events = rag_agent.stream(
        {"messages": messages},
        stream_mode="values",
    )
    
    final_response = ""
    for event in events:
        last_message = event["messages"][-1]
        if hasattr(last_message, 'content') and hasattr(last_message, 'type'):
            if last_message.type == "ai":
                final_response = last_message.content
    
    return final_response

def running_agent():
    """
    Console version of the agent for testing.
    """
    print("\n=== RAG AGENT===")
    
    while True:
        user_input = input("\nWhat is your question: ")
        if user_input.lower() in ['exit', 'quit']:
            break
            
        response = query_agent(user_input)
        print(f"\nAssistant: {response}")

In [57]:
running_agent()


=== RAG AGENT===
Calling Tool: retriever_tool with query: Rahul
Result length: 3840
Tools Execution Complete. Back to the model!

Assistant: Rahul A. Gowda is an AI and machine learning professional with experience in research, development, and application of advanced AI technologies. He is currently a Project Trainee at URSC, ISRO, Bangalore, contributing to the development of generative and agentic AI applications, and conducting research on large language models (LLMs). Rahul's expertise encompasses building computer vision models, creating innovative machine learning and AI tools, and developing phishing detection systems.

He holds a Bachelor of Engineering in Artificial Intelligence from Vemana Institute of Technology, Bengaluru, Karnataka, and has interned in organizations such as Aspire Technologies, Al-Zira Technologies, and Unifirst Robotics. He actively participates in hackathons, demonstrating leadership skills and excelling in technical challenges, with victories in compe